In [ ]:
# TODO: Name of the dataset usually matches the script name with CamelCase instead of snake_case
class NewDataset(datasets.GeneratorBasedBuilder):
    """TODO: Short description of my dataset."""

    VERSION = datasets.Version("1.1.0")

    # This is an example of a dataset with multiple configurations.
    # If you don't want/need to define several sub-sets in your dataset,
    # just remove the BUILDER_CONFIG_CLASS and the BUILDER_CONFIGS attributes.

    # If you need to make complex sub-parts in the datasets with configurable options
    # You can create your own builder configuration class to store attribute, inheriting from datasets.BuilderConfig
    # BUILDER_CONFIG_CLASS = MyBuilderConfig

    # You will be able to load one or the other configurations in the following list with
    # data = datasets.load_dataset('my_dataset', 'first_domain')
    # data = datasets.load_dataset('my_dataset', 'second_domain')
    BUILDER_CONFIGS = [
        datasets.BuilderConfig(name="first_domain", version=VERSION, description="This part of my dataset covers a first domain"),
        datasets.BuilderConfig(name="second_domain", version=VERSION, description="This part of my dataset covers a second domain"),
    ]

    DEFAULT_CONFIG_NAME = "first_domain"  # It's not mandatory to have a default configuration. Just use one if it make sense.

    def _info(self):
        # TODO: This method specifies the datasets.DatasetInfo object which contains informations and typings for the dataset
        if self.config.name == "first_domain":  # This is the name of the configuration selected in BUILDER_CONFIGS above
            features = datasets.Features(
                {
                    "sentence": datasets.Value("string"),
                    "option1": datasets.Value("string"),
                    "answer": datasets.Value("string")
                    # These are the features of your dataset like images, labels ...
                }
            )
        else:  # This is an example to show how to have different features for "first_domain" and "second_domain"
            features = datasets.Features(
                {
                    "sentence": datasets.Value("string"),
                    "option2": datasets.Value("string"),
                    "second_domain_answer": datasets.Value("string")
                    # These are the features of your dataset like images, labels ...
                }
            )
        return datasets.DatasetInfo(
            # This is the description that will appear on the datasets page.
            description=_DESCRIPTION,
            # This defines the different columns of the dataset and their types
            features=features,  # Here we define them above because they are different between the two configurations
            # If there's a common (input, target) tuple from the features, uncomment supervised_keys line below and
            # specify them. They'll be used if as_supervised=True in builder.as_dataset.
            # supervised_keys=("sentence", "label"),
            # Homepage of the dataset for documentation
            homepage=_HOMEPAGE,
            # License for the dataset if available
            license=_LICENSE,
            # Citation for the dataset
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        # TODO: This method is tasked with downloading/extracting the data and defining the splits depending on the configuration
        # If several configurations are possible (listed in BUILDER_CONFIGS), the configuration selected by the user is in self.config.name

        # dl_manager is a datasets.download.DownloadManager that can be used to download and extract URLS
        # It can accept any type or nested list/dict and will give back the same structure with the url replaced with path to local files.
        # By default the archives will be extracted and a path to a cached folder where they are extracted is returned instead of the archive
        urls = _URLS[self.config.name]
        data_dir = dl_manager.download_and_extract(urls)
        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "train.jsonl"),
                    "split": "train",
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.VALIDATION,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "dev.jsonl"),
                    "split": "dev",
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.TEST,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "test.jsonl"),
                    "split": "test"
                },
            ),
        ]

    # method parameters are unpacked from `gen_kwargs` as given in `_split_generators`
    def _generate_examples(self, filepath, split):
        # TODO: This method handles input defined in _split_generators to yield (key, example) tuples from the dataset.
        # The `key` is for legacy reasons (tfds) and is not important in itself, but must be unique for each example.
        with open(filepath, encoding="utf-8") as f:
            for key, row in enumerate(f):
                data = json.loads(row)
                if self.config.name == "first_domain":
                    # Yields examples as (key, example) tuples
                    yield key, {
                        "sentence": data["sentence"],
                        "option1": data["option1"],
                        "answer": "" if split == "test" else data["answer"],
                    }
                else:
                    yield key, {
                        "sentence": data["sentence"],
                        "option2": data["option2"],
                        "second_domain_answer": "" if split == "test" else data["second_domain_answer"],
                    }

# Dataset creation. Run during first time setup

In [3]:
import os
import string
import csv
from fontpreview import FontPreview
import pandas as pd
import requests
import io
from zipfile import ZipFile
import json
import shutil
import uuid
from tqdm.notebook import tqdm
from datasets import load_dataset, Image
from fontTools.ttLib import TTFont
from fontTools.unicode import Unicode


def has_glyph(font, glyph):
    for table in font['cmap'].tables:
        if ord(glyph) in table.cmap.keys():
            return True
    return False

#For a given font file, create the alphabet and the numbers 0-9
def create_alphabet(font_file, image_folder):
    font = FontPreview(font_file)
    ttf_font = TTFont(font_file)
    font_name = font.font.getname()[0]
    included_chars = []
    for char in string.ascii_letters:
        if has_glyph(ttf_font, char):
            included_chars.append(char)
    for char in string.digits:
        if has_glyph(ttf_font, char):
            included_chars.append(char)
    split_folder = 'train'
    if len(included_chars) != 62:
        split_folder = 'test'
        
        
    save_path = os.path.join(image_folder, split_folder, font_name)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for char in included_chars:
        font.font_text = char
        font.bg_color = (0, 0, 0)  # white BG
        font.dimension = (512, 512)  # Dimension consistent with the default resolution for diffusion models
        font.fg_color = (255, 255, 255)  # Letter color
        font.set_font_size(300)  # font size ~ 300 pixels
        font.set_text_position('center')  # center placement

        if char in string.ascii_lowercase:
            image_file_name = 'lower_' + char + '.jpg'
        elif char in string.ascii_uppercase:
            image_file_name = 'upper_' + char + '.jpg'
        else:
            image_file_name = char + '.jpg'
        font.save(os.path.join(save_path, image_file_name))

def create_alphabet_for_each_ttf():
    TTF_DIR = os.path.join(os.path.abspath(os.getcwd()), 'ttf-files')
    IMG_DIR = os.path.join(os.path.abspath(os.getcwd()), 'font-images')
    if not os.path.exists(IMG_DIR):
        os.mkdir(IMG_DIR)
    fnames = os.listdir(TTF_DIR)

    for fname in tqdm(fnames):
        TTF_PATH = os.path.join(TTF_DIR, fname)
        create_alphabet(TTF_PATH, IMG_DIR)

    

#Uses pandas to read through the CSV from sheets without the need of constantly redownloading
def get_font_ttfs():
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv('font_dataset.csv')
    # Create data folder if it does not exist
    if not os.path.exists('ttf-files'):
        os.makedirs('ttf-files')
    # Loop through each row of the DataFrame
    for index, row in tqdm(df.iterrows()):
        # Get the link and filename for the current row
        link = row['Link']
        filename = row['Filename']
        if os.path.exists(os.path.join('ttf-files', filename)):
            continue
            
        
        # Download the zip file from the link
        response = requests.get(link, stream=True)
        with open('temp.zip', 'wb') as temp_file:
            shutil.copyfileobj(response.raw, temp_file)
        del response
        
        # Unzip the downloaded file
        with ZipFile('temp.zip', 'r') as zip_file:
            zip_file.extract(filename)
            
        # Move the file to the data folder
        source_path = os.path.join(os.getcwd(), filename)
        dest_path = os.path.join(os.getcwd(), 'ttf-files', filename)
        shutil.move(source_path, dest_path)
        
        # Remove the temporary zip file
        os.remove('temp.zip')


#Create the jsonl file and training folder for the images
def create_dataset():
    FONT_IMAGE_PATH = os.path.join(os.getcwd(), 'font-images')
    assert os.path.exists(FONT_IMAGE_PATH)
    TTF_PATH = os.path.join(os.getcwd(), 'ttf-files')
    assert os.path.exists(TTF_PATH)
    CSV_PATH = os.path.join(os.getcwd(), 'font_dataset.csv')

    
    # Step 1: Initialize the json file
    # Step 2: Loop through the Dataframe, for each row the Filename column corresponds to the actual
    #         folder name in 'font_images'.
    # Step 3: For each image in the respective folder, copy it over to the training folder (renaming it) and add its entry
    #         to the jsonl file

    #Step 1
    json_metadata = []
    # if not os.path.exists(training_data_path):
    #     os.makedirs(training_data_path)


    #Step 2
    df = pd.read_csv(CSV_PATH)
    train_dataset = []
    test_dataset = []
    head = df.head()
    for idx, row_data in df.iterrows():
        ttf_path = os.path.join(TTF_PATH, row_data['Filename'])
        font_img_dir = FontPreview(ttf_path).font.getname()[0]
        split_folder = 'train'
        font_img_dir_path = os.path.join(FONT_IMAGE_PATH, split_folder, font_img_dir)
        if not os.path.exists(font_img_dir_path):
            split_folder = 'test'
            font_img_dir_path = os.path.join(FONT_IMAGE_PATH, split_folder, font_img_dir)
        font_img_paths = [os.path.join(font_img_dir_path, fname) for fname in os.listdir(font_img_dir_path)]
        included_chars = [cur_img_path.split('/')[-1].split('.')[0] for cur_img_path in font_img_paths]
        json_data_row = {
            'uniqueId': str(uuid.uuid4()),
            'font_img_paths': font_img_paths,
            'ttf_path': ttf_path,
            'font_characteristics': row_data['Descriptors'], 
            'chars': included_chars,
            'font_properties': {
                'font_weight': row_data['Weight'], 
                'rounding': row_data['Courner Rounding'], 
                'font_serifs': row_data['Serif'],
                'width': row_data['Width'],
                'capitals': row_data['Capitals'],
                'dynamics': row_data['Dynamics'] 
            }
        }
        if split_folder == 'train':
            train_dataset.append(json_data_row)
        else:
            test_dataset.append(json_data_row) 
    #Create the jsonl file
    with open('train-metadata.jsonl', 'w') as f:
        for item in train_dataset:
            f.write(json.dumps(item) + '\n')
    with open('test-metadata.jsonl', 'w') as f:
        for item in test_dataset:
            f.write(json.dumps(item) + '\n')




# if __name__ == '__main__':
    #get_fonts('font_files', 'font_images')
    #create_dataset('font_images', 'font_files', 'train')


In [ ]:
create_dataset()

In [ ]:
get_font_ttfs()

In [ ]:
# shouldnt run this on TACC, best to scp the files from your own computer
# scp localhost/font-diffusion/font-images.tar <your_user>@maverick2.tacc.utexas.edu:\$WORK/font-diffusion
create_alphabet_for_each_ttf()

In [5]:
import os
import string
import csv
from fontpreview import FontPreview
import pandas as pd
import requests
import io
from zipfile import ZipFile
import json
import shutil
import uuid
from tqdm.notebook import tqdm
from datasets import load_dataset, Image
from fontTools.ttLib import TTFont
from fontTools.unicode import Unicode


def has_glyph(font, glyph):
    for table in font['cmap'].tables:
        if ord(glyph) in table.cmap.keys():
            return True
    return False

#For a given font file, create the alphabet and the numbers 0-9
def create_alphabet(font_file, image_folder):
    font = FontPreview(font_file)
    ttf_font = TTFont(font_file)
    font_name = font.font.getname()[0]
    included_chars = []
    for char in string.ascii_letters:
        if has_glyph(ttf_font, char):
            included_chars.append(char)
    for char in string.digits:
        if has_glyph(ttf_font, char):
            included_chars.append(char)
    split_folder = 'train'
    if len(included_chars) != 62:
        split_folder = 'test'
        
        
    save_path = os.path.join(image_folder, split_folder, font_name)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for char in included_chars:
        font.font_text = char
        font.bg_color = (0, 0, 0)  # white BG
        font.dimension = (512, 512)  # Dimension consistent with the default resolution for diffusion models
        font.fg_color = (255, 255, 255)  # Letter color
        font.set_font_size(300)  # font size ~ 300 pixels
        font.set_text_position('center')  # center placement

        if char in string.ascii_lowercase:
            image_file_name = 'lower_' + char + '.jpg'
        elif char in string.ascii_uppercase:
            image_file_name = 'upper_' + char + '.jpg'
        else:
            image_file_name = char + '.jpg'
        font.save(os.path.join(save_path, image_file_name))

def create_alphabet_for_each_ttf():
    TTF_DIR = os.path.join(os.path.abspath(os.getcwd()), 'ttf-files')
    IMG_DIR = os.path.join(os.path.abspath(os.getcwd()), 'font-images')
    if not os.path.exists(IMG_DIR):
        os.mkdir(IMG_DIR)
    fnames = os.listdir(TTF_DIR)

    for fname in tqdm(fnames):
        TTF_PATH = os.path.join(TTF_DIR, fname)
        create_alphabet(TTF_PATH, IMG_DIR)

    

#Uses pandas to read through the CSV from sheets without the need of constantly redownloading
def get_font_ttfs():
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv('font_dataset.csv')
    # Create data folder if it does not exist
    if not os.path.exists('ttf-files'):
        os.makedirs('ttf-files')
    # Loop through each row of the DataFrame
    for index, row in tqdm(df.iterrows()):
        # Get the link and filename for the current row
        link = row['Link']
        filename = row['Filename']
        if os.path.exists(os.path.join('ttf-files', filename)):
            continue
            
        
        # Download the zip file from the link
        response = requests.get(link, stream=True)
        with open('temp.zip', 'wb') as temp_file:
            shutil.copyfileobj(response.raw, temp_file)
        del response
        
        # Unzip the downloaded file
        with ZipFile('temp.zip', 'r') as zip_file:
            zip_file.extract(filename)
            
        # Move the file to the data folder
        source_path = os.path.join(os.getcwd(), filename)
        dest_path = os.path.join(os.getcwd(), 'ttf-files', filename)
        shutil.move(source_path, dest_path)
        
        # Remove the temporary zip file
        os.remove('temp.zip')


#Create the jsonl file and training folder for the images
def create_dataset():
    FONT_IMAGE_PATH = os.path.join(os.getcwd(), 'font-images')
    assert os.path.exists(FONT_IMAGE_PATH)
    TTF_PATH = os.path.join(os.getcwd(), 'ttf-files')
    assert os.path.exists(TTF_PATH)
    CSV_PATH = os.path.join(os.getcwd(), 'font_dataset.csv')

    
    # Step 1: Initialize the json file
    # Step 2: Loop through the Dataframe, for each row the Filename column corresponds to the actual
    #         folder name in 'font_images'.
    # Step 3: For each image in the respective folder, copy it over to the training folder (renaming it) and add its entry
    #         to the jsonl file

    #Step 1
    json_metadata = []
    # if not os.path.exists(training_data_path):
    #     os.makedirs(training_data_path)


    #Step 2
    df = pd.read_csv(CSV_PATH)
    train_dataset = []
    test_dataset = []
    head = df.head()
    for idx, row_data in df.iterrows():
        ttf_path = os.path.join(TTF_PATH, row_data['Filename'])
        font_img_dir = FontPreview(ttf_path).font.getname()[0]
        split_folder = 'train'
        font_img_dir_path = os.path.join(FONT_IMAGE_PATH, split_folder, font_img_dir)
        if not os.path.exists(font_img_dir_path):
            split_folder = 'test'
            font_img_dir_path = os.path.join(FONT_IMAGE_PATH, split_folder, font_img_dir)
        font_img_paths = [os.path.join(font_img_dir_path, fname) for fname in os.listdir(font_img_dir_path)]
        included_chars = [cur_img_path.split('/')[-1].split('.')[0] for cur_img_path in font_img_paths]
        json_data_row = {
            'uniqueId': str(uuid.uuid4()),
            'font_img_paths': font_img_paths,
            'ttf_path': ttf_path,
            'font_characteristics': row_data['Descriptors'], 
            'chars': included_chars,
            'font_properties': {
                'font_weight': row_data['Weight'], 
                'rounding': row_data['Courner Rounding'], 
                'font_serifs': row_data['Serif'],
                'width': row_data['Width'],
                'capitals': row_data['Capitals'],
                'dynamics': row_data['Dynamics'] 
            }
        }
        if split_folder == 'train':
            train_dataset.append(json_data_row)
        else:
            test_dataset.append(json_data_row) 
    #Create the jsonl file
    with open('train-metadata.jsonl', 'w') as f:
        for item in train_dataset:
            f.write(json.dumps(item) + '\n')
    with open('test-metadata.jsonl', 'w') as f:
        for item in test_dataset:
            f.write(json.dumps(item) + '\n')




# if __name__ == '__main__':
    #get_fonts('font_files', 'font_images')
    #create_dataset('font_images', 'font_files', 'train')


Found cached dataset json (/home1/08823/msrodlab/.cache/huggingface/datasets/json/default-13b631bc655bda2c/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['uniqueId', 'font_img_paths', 'ttf_path', 'font_characteristics', 'chars', 'font_properties'],
        num_rows: 196
    })
    test: Dataset({
        features: ['uniqueId', 'font_img_paths', 'ttf_path', 'font_characteristics', 'chars', 'font_properties'],
        num_rows: 16
    })
})


In [6]:
!which python3

/work/08823/msrodlab/miniconda3/bin/python3
